## Import Module

In [ ]:
# *------- Basic setup -------*
import numpy as np
import pandas as pd
import os, random, time
import copy
from tqdm.notebook import tqdm
from multiprocessing import cpu_count
import matplotlib.pyplot as plt
from PIL import Image

# *------- torch -------*
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchsummary import summary

from torchvision import transforms, models

# *------- albumentations -------*
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# *------- sklearn -------*
from sklearn.model_selection import train_test_split

## Configuration

In [ ]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

SEED = 42
set_seed(SEED)

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
IMG_SIZE = (64, 64)
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_CLASSES = 256
NUM_EPOCHS = 100
NUM_CPU = cpu_count()

print(DEVICE)

## Split Dataset

In [ ]:
def split_dataset(csv_path, train_save_path, valid_save_path, test_size=0.2, random_seed=SEED):
    """
    주어진 CSV 파일을 train과 val 데이터셋으로 나누어 저장하는 함수.

    Args:
    - csv_path: 입력 CSV 파일 경로
    - train_save_path: 학습 데이터셋 저장 경로
    - valid_save_path: 검증 데이터셋 저장 경로
    - test_size: 검증 데이터셋의 비율 (default: 0.2)
    - random_seed: 데이터 분할시 사용되는 랜덤 시드 (default: 42)

    Returns:
    - None
    """
    
    # CSV 파일 읽기
    df = pd.read_csv(csv_path, header=None, names=['path', 'label'])

    # 데이터를 train과 val로 나누기
    train_df, valid_df = train_test_split(df, test_size=test_size, random_state=random_seed, stratify=df['label'])

    # 나눠진 데이터를 CSV 파일로 저장
    train_df.to_csv(train_save_path, index=False, header=False)
    valid_df.to_csv(valid_save_path, index=False, header=False)


In [ ]:
split_dataset('image-data/labels-map.csv', 'image-data/train-labels.csv', 'image-data/valid-labels.csv')

## Dataset

In [ ]:
class KoreanHandwritingDataset(Dataset):
    """
    한글 손글씨 데이터셋을 로드하고 처리하기 위한 클래스.

    Attributes:
    - dataset (DataFrame): CSV 파일에서 읽어들인 데이터.
    - root_dir (str): 이미지 파일이 저장된 기본 디렉터리 경로.
    - transform (callable, optional): 샘플에 적용될 변환 (예: 데이터 증강).
    """
    
    def __init__(self, csv_file, root_dir, label_file, transform=None):
        """
        Parameters:
        - csv_file (str): CSV 파일의 경로.
        - root_dir (str): 모든 이미지가 저장된 디렉터리 경로.
        - label_file (str) :
        - transform (callable, optional): 샘플에 적용할 선택적 변환.
        """
        self.dataset = pd.read_csv(csv_file, header=None, names=['path', 'label'])
        self.root_dir = root_dir
        self.label_file = label_file
        self.transform = transforms

        # 파일에서 한글 글자를 읽어오기
        with open(self.label_file, 'r', encoding='utf-8') as f:
            hangul_chars = [line.strip() for line in f.readlines()]
        
        # 각 한글 글자에 순차적으로 레이블 번호 부여
        self.label_mapping = {char: idx for idx, char in enumerate(hangul_chars)}

    def __len__(self):
        """
        데이터셋의 길이를 반환합니다.
        
        Returns:
        - int: 데이터셋 내의 샘플 수.
        """
        return len(self.dataset)

    def __getitem__(self, idx):
        """
        주어진 인덱스에 해당하는 샘플을 반환합니다.
        
        Parameters:
        - idx (int): 반환할 샘플의 인덱스.

        Returns:
        - dict: 'image' 및 'label' 키를 포함하는 사전.
        """
        img_path = os.path.join(self.root_dir, self.dataset.iloc[idx]['path'])
        # Gray 이미지를 RGB로 열기
        img = Image.open(img_path).convert("RGB")
        img = np.array(img, dtype=np.float32)
        img /= 255
        
        if self.transforms:
            img = self.transform(image=img)['image']
                
        label = self.dataset.iloc[idx]['label']
        label = self.label_mapping[label]
        label = torch.tensor(label, dtype=torch.int64)
        return img, label
    

In [ ]:
TRAIN_CSV_FILE = "./image-data/train-labels.csv"
VALID_CSV_FILE = "./image-data/valid-labels.csv"
LABEL_FILE = "./labels/256-common-hangul.txt"
ROOT_DIR = "./image-data/hangul-images"

train_dataset = KoreanHandwritingDataset(TRAIN_CSV_FILE, ROOT_DIR, LABEL_FILE)
valid_dataset = KoreanHandwritingDataset(VALID_CSV_FILE, ROOT_DIR, LABEL_FILE)

train_size = len(train_dataset)
valid_size = len(valid_dataset)
print(train_size, valid_size)


In [ ]:

# 채널 별 mean 계산
def get_mean(dataset):
  meanRGB = [np.mean(image.numpy(), axis=(1,2)) for image,_ in dataset]
  meanR = np.mean([m[0] for m in meanRGB])
  meanG = np.mean([m[1] for m in meanRGB])
  meanB = np.mean([m[2] for m in meanRGB])
  return [meanR, meanG, meanB]

# 채널 별 str 계산
def get_std(dataset):
  stdRGB = [np.std(image.numpy(), axis=(1,2)) for image,_ in dataset]
  stdR = np.mean([s[0] for s in stdRGB])
  stdG = np.mean([s[1] for s in stdRGB])
  stdB = np.mean([s[2] for s in stdRGB])
  return [stdR, stdG, stdB]

In [ ]:

train_transforms = transforms.Compose(
    [transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(get_mean(train_dataset), get_std(train_dataset))]
    )
valid_transforms = transforms.Compose(
    [transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(get_mean(test_dataset), get_std(test_dataset))]
    )

train_dataset.transform = train_transforms
valid_dataset.transform = valid_transforms

In [ ]:
# dataloader 생성
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers = NUM_CPU)
valid_loader = DataLoader(valid_dataset, batch_size = BATCH_SIZE, shuffle=False, num_workers = NUM_CPU)

## Model

In [ ]:
# 사전학습된 가중치를 가져오도록 pretrained는 True로 설정
model = models.resnet34(pretrained=True)

# number of features in the input of the linear layer
num_ftrs = model.fc.in_features

# sets the number of features of the linear layer
model.fc = torch.nn.Linear(num_ftrs, NUM_CLASSES)
model = model.to(DEVICE)

# parameters
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

summary(model, (3, 64, 64))


In [ ]:
# 모델 학습 함수 정의
def train_model(model, criterion, optimizer, num_epochs, train_loader, val_loader, save_dir='model_best.pth'):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        
        # 학습 모드 설정
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        for step, (inputs, labels) in enumerate(tqdm(train_loader, desc="train")):
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
        print('Train Loss: {:.4f} Train Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        # 검증 모드 설정
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in tqdm(val_loader, desc="valid"):
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            optimizer.zero_grad()
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / valid_size
        epoch_acc = running_corrects.double() / valid_size
        print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        print('-' * 30)
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model.state_dict())
            # 최적의 모델 가중치 저장
            torch.save(best_model, save_dir)
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Val Acc: {:.4f}'.format(best_acc))
    model.load_state_dict(best_model)
    return model


In [ ]:

# train the model
model = train_model(model, criterion, optimizer, NUM_EPOCHS, train_loader, valid_loader)

In [ ]:


def predict_image(model, image_path, transform):
    # 이미지 불러오기
    image = Image.open(image_path).convert("RGB")
    
    # 이미지 전처리
    image = transform(image).unsqueeze(0) # 차원 추가 (배치 차원)
    image = image.to(DEVICE)
    
    # 모델 추론 모드 설정 및 예측
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        return predicted.item()

# ResNet34의 입력 사이즈와 일치하도록 이미지 전처리를 위한 transform 정의
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),  # ResNet34 입력 사이즈에 맞춰 이미지 크기 조정
    transforms.ToTensor(),
    # 만약 추가 전처리가 필요하면 이곳에 추가
])



In [ ]:
# 예측 수행
image_path = './test-images/test_1.jpeg'
predicted_label = predict_image(model, image_path, transform)
print(f"Predicted Label: {predicted_label}")


In [ ]:
image_path = './test-images/test_2.jpg'
predicted_label = predict_image(model, image_path, transform)
print(f"Predicted Label: {predicted_label}")

In [ ]:
image_path = './test-images/test_3.jpg'
predicted_label = predict_image(model, image_path, transform)
print(f"Predicted Label: {predicted_label}")